In [45]:
import pandas as pd
import asyncio
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [69]:
import re

In [46]:
url = "https://ews.tropmet.res.in/"

In [47]:
await page.goto(url, wait_until='networkidle', timeout=0)
await page.locator("div.slideshow-container").wait_for()

In [65]:
await page.locator("div.slideshow-container strong").all_text_contents()

['Current AQI - 2672025-01-28 01',
 'Forecast AQI - 1652025-01-28 07',
 'Current AQI - 1052024-12-31 23',
 'Forecast AQI - 812025-01-28 07',
 'Current AQI - 1432025-01-22 09',
 'Forecast AQI - 1942025-01-26 07',
 'Current AQI - 1612023-12-31 23',
 'Forecast AQI - 3612025-01-25 07',
 'Current AQI - 1332025-01-23 14',
 'Forecast AQI -402025-01-25 07',
 'Current AQI - 02025-01-23 14',
 'Forecast AQI - 232025-01-28 08',
 'Current AQI -2232025-01-22 09',
 'Forecast AQI -2202025-01-25 07',
 'Current AQI - 842025-01-22 09',
 'Forecast AQI - 912025-01-25 07',
 'Current AQI - 622024-07-11 16',
 'Forecast AQI - 792025-01-26 07']

In [58]:
dates = await page.locator("div.slideshow-container strong").all_text_contents()

In [68]:
dates[2]

'Current AQI - 1052024-12-31 23'

In [70]:
date = [re.search(r'\d{4}-\d{2}-\d{2}', item).group() for item in dates]

In [71]:
date


['2025-01-28',
 '2025-01-28',
 '2024-12-31',
 '2025-01-25',
 '2025-01-22',
 '2025-01-25',
 '2023-12-31',
 '2025-01-28',
 '2025-01-23',
 '2025-01-28',
 '2025-01-23',
 '2025-01-26',
 '2025-01-22',
 '2025-01-28',
 '2025-01-22',
 '2025-01-28',
 '2024-07-11',
 '2025-01-25']

In [97]:
date_result = [elem for index, elem in enumerate(date) if (index + 1) % 2 != 0]
on_date

['2025-01-28',
 '2024-12-31',
 '2025-01-22',
 '2023-12-31',
 '2025-01-23',
 '2025-01-23',
 '2025-01-22',
 '2025-01-22',
 '2024-07-11']

In [42]:
# await browser.close()

In [28]:
result = await page.locator("text tspan.highcharts-text-outline").all_text_contents()

In [29]:
result

['267\u200b',
 '177\u200b',
 '105\u200b',
 '81\u200b',
 '143\u200b',
 '173\u200b',
 '161\u200b',
 '336\u200b',
 '133\u200b',
 '50\u200b',
 '0\u200b',
 '45\u200b',
 '223\u200b',
 '116\u200b',
 '84\u200b',
 '99\u200b',
 '62\u200b',
 '97\u200b']

In [30]:
All = [element.replace("\u200b", "") for element in result]
All

['267',
 '177',
 '105',
 '81',
 '143',
 '173',
 '161',
 '336',
 '133',
 '50',
 '0',
 '45',
 '223',
 '116',
 '84',
 '99',
 '62',
 '97']

In [104]:
aqi = [elem for index, elem in enumerate(All) if (index + 1) % 2 != 0]
aqi

['267', '105', '143', '161', '133', '0', '223', '84', '62']

In [94]:
cities = ['Delhi', 'Pune', 'Mumbai', 'Kolkata', 'Bengalore', 'Lucknow', 'Patna', 'Hyderabad', 'Ahmedabad']

In [99]:
result_date_aqi = [{"aqi_result": aqi, "date_result": dt, "cities": city} for aqi, dt, city in zip(aqi_result, date_result, cities)]
result_date_aqi

[{'aqi_result': '267', 'date_result': '2025-01-28', 'cities': 'Delhi'},
 {'aqi_result': '105', 'date_result': '2024-12-31', 'cities': 'Pune'},
 {'aqi_result': '143', 'date_result': '2025-01-22', 'cities': 'Mumbai'},
 {'aqi_result': '161', 'date_result': '2023-12-31', 'cities': 'Kolkata'},
 {'aqi_result': '133', 'date_result': '2025-01-23', 'cities': 'Bengalore'},
 {'aqi_result': '0', 'date_result': '2025-01-23', 'cities': 'Lucknow'},
 {'aqi_result': '223', 'date_result': '2025-01-22', 'cities': 'Patna'},
 {'aqi_result': '84', 'date_result': '2025-01-22', 'cities': 'Hyderabad'},
 {'aqi_result': '62', 'date_result': '2024-07-11', 'cities': 'Ahmedabad'}]

In [101]:
df = pd.DataFrame(result_date_aqi)

In [102]:
df.to_csv("major_cities_aqi.csv", index=False)

In [103]:
pd.read_csv("major_cities_aqi.csv")

,aqi_result,date_result,cities
0,267,2025-01-28,Delhi
1,105,2024-12-31,Pune
2,143,2025-01-22,Mumbai
3,161,2023-12-31,Kolkata
4,133,2025-01-23,Bengalore
5,0,2025-01-23,Lucknow
6,223,2025-01-22,Patna
7,84,2025-01-22,Hyderabad
8,62,2024-07-11,Ahmedabad


# New York

In [31]:
import pandas as pd
import asyncio
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [32]:
url = "https://docs.airnowapi.org/feeds"

In [33]:
await page.goto(url)

<Response url='https://docs.airnowapi.org/login;jsessionid=9DC7BC26893BAB081F1CB70A7754A4B8?index=' request=<Request url='https://docs.airnowapi.org/login;jsessionid=9DC7BC26893BAB081F1CB70A7754A4B8?index=' method='GET'>>

In [40]:
await page.get_by_label("User Name:").fill("shalaka")

In [41]:
await page.get_by_label("Password:").fill("Data@2023")

In [44]:
await page.get_by_role("button").click()

In [58]:
await page.locator("#code").select_option("NY")

['NY']

In [61]:
await page.get_by_role("button", name="Go").click()

In [69]:
await page.inner_text("New York City Region")

TimeoutError: Timeout 30000ms exceeded.
=========================== logs ===========================
waiting for locator("New York City Region")
============================================================